# Web scraping com Python

Este notebook demonstra como você pode usar a linguagem de programação Python para extrair informações de uma página da web. O objetivo hoje: raspar a tabela principal [a primeira página da lista de cartas WARN (notificação de ajuste de trabalho e reciclagem) de Maryland](https://www.dllr.state.md.us/employment/warn.shtml) e, se houver tempo, gravar os dados em um CSV.

Se você é relativamente novo no Python, pode ser útil manter [este notebook de dicas da sintaxe do Python](https://github.com/abraji/cursos_NICAR20/blob/master/python_ire/Python%20101.ipynb) aberto em outra guia enquanto você trabalha neste caderno.

### Índice

- [Using Jupyter notebooks](#Usando-o-Jupyter-notebooks)
- [What _is_ a web page, anyway?](#O-que-é-uma-página-da-Web,-afinal?)
- [Inspect the source](#Inspecione-a-fonte)
- [Import libraries](#Importar-bibliotecas)
- [Request the page](#Solicite-a-página)
- [Turn your HTML into soup](#Transforme-seu-HTML-em-sopa)
- [Targeting and extracting data](#Segmentação-e-extração-de-dados)
- [Write the results to file](#Escreva-os-resultados-no-arquivo)

### Usando o Jupyter notebooks

Existem várias maneiras de escrever e executar o código Python no seu computador. Uma maneira - o método que estamos usando hoje - é usar [Jupyter notebooks](https://jupyter.org/), que são executados no seu navegador e permitem intercalar a documentação com o seu código. Eles são úteis para agrupar seu código com uma explicação legível por humanos do que está acontecendo em cada etapa. Confira alguns exemplos do [L.A. Times](https://github.com/datadesk/notebooks) e [BuzzFeed News](https://github.com/BuzzFeedNews/everything#data-and-analyses).

**Para adicionar uma nova célula ao seu notebook**: Clique no botão + no menu.

**Para executar uma célula de código**: Selecione a célula e clique no botão "Executar" no menu ou pressione Shift + Enter.

**Uma pegadinha comum**: O notebook não "sabe" sobre o código que você escreveu até ter um _executar (run)_ a célula em que está o código. Por exemplo, se você definir uma variável chamada `my_name` em uma célula, e depois, quando você tenta acessar essa variável em outra célula, mas obtém um erro que diz `NameError: name 'my_name' is not defined`, a solução mais provável é executar (ou executar novamente) a célula na qual você definiu `my_name`.

### O que é uma página da Web, afinal?

Geralmente, uma página da web consiste em vários arquivos de texto formatados especificamente armazenados em um computador (um _servidor_) que provavelmente está em um rack em um gigantesco data center em algum lugar.

Principalmente você estará lidando com arquivos `.html` (HyperText Markup Language - Linguagem de marcação de hipertexto) que podem incluir referências a arquivos `.css` (Cascading Style Sheet - Planilha em estilo cascata), que determinam a aparência da página, e/ou arquivos `.js` (JavaScript), que adicionam interatividade e outros arquivos de texto especialmente formatados.

Hoje, vamos nos concentrar no HTML, que estrutura a página.

A maioria dos elementos HTML é representada por um par de tags - uma tag de abertura e uma tag de fechamento.

Uma tabela, por exemplo, começa com `<table>` e termina com `</table>`. A primeira tag informa ao navegador: "Ei! Eu tenho uma mesa aqui! Renderize-a como uma mesa." A tag de fechamento (observe a barra!) Informa ao navegador: "Ei! Estou pronto com essa tabela, obrigado." Dentro da tabela estão aninhadas mais tags HTML que representam linhas (`<tr>`) e células (`<td>`).

Os elementos HTML podem ter qualquer número de atributos, como classes --

`<table class="cool-table">`

-- estilos --

`<table style="width:95%;">`

-- hiperlinks para outras páginas --

`<a href="https://ire.org">Click here to visit IRE's website</a>`

-- e IDs --

`<table id="cool-table">`

-- isso será útil saber quando estamos raspando.

### Inspecione a fonte

Você pode olhar o HTML que compõe uma página da Web com o comando _Inspecionar Elemento_ em um navegador da web. Nós gostamos do Chrome e Firefox para isso; hoje, usaremos o Chrome.

Você pode inspecionar elementos específicos da página clicando com o botão direito do mouse na página e selecionando "Inspecionar" or "Inspecionar Elemento" no menu pop-up que aparece. Passe o mouse sobre os elementos de interesse na aba "Inspetor" para destacá-los na página.

Para examinar todo o código-fonte que compõe uma página, você pode clicar em "Ver código-fonte da página". No Chrome, um atalho é pressionar `Ctrl+U` em um PC ou `⌘+Opt+U` em um Mac. (Também está na barra de menus: Exibir> Desenvolvedor> Exibir fonte da página.)

Você receberá uma página mostrando todo o código HTML que compõe essa página. Ignore 99% e tente localizar o (s) elemento (s) que deseja explorar (use `Ctrl+F` em um PC e `⌘+F` no Mac para procurar).

Abra um navegador Chrome e inspecione a tabela na [primeira página da lista de cartas WARN (notificação de ajuste de trabalho e reciclagem) de Maryland](https://www.dllr.state.md.us/employment/warn.shtml). Encontre a tabela que queremos raspar.

É a única tabela na página? Caso contrário, ela possui algum atributo que permita segmentá-la?

### Instalação
Uma biblioteca de terceiros instalada separadamente do Python, como é o caso do pandas, pode ser instalada de várias formas - a depender de como você usa o Python. Veja mais [aqui](https://pandas.pydata.org/docs/getting_started/install.html)

No terminal um modo muito comum é usar o pip: `pip install beautifulsoup4`
`pip install requests`

### Importar bibliotecas

O passo um é _importar_ duas bibliotecas Python de terceiros que nos ajudarão a raspar esta página:
- `requests` é o padrão de fato para fazer solicitações HTTP, semelhante ao que acontece quando você digita um URL em uma janela do navegador e pressiona enter.
- `bs4`, ou BeautifulSoup, é uma biblioteca popular para analisar HTML em uma estrutura de dados com a qual o Python pode trabalhar.

Essas bibliotecas são instaladas separadamente do Python por projeto - ([leia mais sobre nossas recomendações para configurar projetos Python aqui](https://docs.google.com/document/d/1cYmpfZEZ8r-09Q6Go917cKVcQk_d0P61gm0q8DAdIdg/edit#heading=h.od2v1nkge5t1)).

Execute esta célula (você só precisará fazer isso uma vez):

In [3]:
import requests
import bs4

### Solicite a página

Em seguida, usaremos o método `get()` da biblioteca `requests` (que acabamos de importar) para acessar a página da web.

Enquanto estamos nisso, vamos atribuir todas as coisas que retornam a uma nova variável usando `=`.

O nome da variável é arbitrário, mas geralmente é bom escolher algo que descreva o valor para o qual está apontando.

Observe que o URL que estamos pegando está entre aspas, tornando-o uma string que o Python interpretará como texto (em oposição a números, booleanos, etc.). Você pode ler mais sobre tipos de dados Python e atribuição de variáveis [aqui](https://github.com/abraji/cursos_NICAR20/blob/master/python_ire/Python%20101.ipynb).

Execute estas duas células:

In [4]:
URL = 'http://www.dllr.state.md.us/employment/warn.shtml'

In [5]:
warn_page = requests.get(URL)

Parece que nada aconteceu, o que é (geralmente) um bom sinal.

Se você quiser ter certeza de que sua solicitação foi bem-sucedida, verifique o atributo `status_code` do objeto Python retornado:

In [ ]:
warn_page.status_code

Um código `200` significa que tudo está bem. `404` significa que a página não foi encontrada etc. ([Aqui está uma de nossas listas favoritas de códigos de status HTTP](https://http.cat/) ([ou aqui, se você prefere cachorros](https://httpstatusdogs.com/)).)

O objeto que está sendo armazenado como a variável `warn_page` voltou com muitas informações potencialmente úteis que podemos acessar. Hoje, estamos mais interessados no atributo `.text` -- o HTML que compõe a página da web, como se tivéssemos visto a fonte da página. Vamos dar uma olhada:

In [ ]:
warn_page.text

### ✍️ Faça você mesmo

Use os blocos de código abaixo para experimentar solicitando páginas da Web e verificando o HTML que é retornado.

Algumas ideias para você começar:
- `'http://ire.org'`
- `'https://web.archive.org/web/20031202214318/http://www.tdcj.state.tx.us:80/stat/finalmeals.htm'`
- `'https://www.nrc.gov/reactors/operating/list-power-reactor-units.html'`

### Transforme seu HTML em sopa

O HTML no atributo`.text` é um objeto da solicitação e é apenas uma string - um grande pedaço de texto.

Antes de começarmos a direcionar e extrair partes de dados no HTML, precisamos transformar esse pedaço de texto em uma estrutura de dados com a qual o Python possa trabalhar. É aí que a biblioteca [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) (`bs4`) entra.

Criaremos uma nova instância de um objeto `BeautifulSoup`, que vive sob o nível superior da biblioteca `bs4` que importamos anteriormente. Precisamos de duas coisas:
- O HTML que gostaríamos de analisar -- `warn_page.text`
- Uma sequência com o nome do tipo de analisador (parser) a ser usado -- `html.parser` é o default e geralmente suficiente, mas [existem outras opções de parser](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#installing-a-parser)

Salvaremos o HTML analisado como uma nova variável, `soup`.

In [ ]:
soup = bs4.BeautifulSoup(warn_page.text, 'html.parser')

Nada aconteceu, o que é bom! Você pode dar uma olhada no que `soup` é, mas parece muito com `warn_page.text`:

In [ ]:
soup

Se você quiser ter certeza, pode usar a função Python `type()` para verificar com que tipo de objeto você está lidando:

In [ ]:
# o tipo `str` significa uma string, ou texto
type(warn_page.text)

In [ ]:
# o tipo `bs4.BeautifulSoup` significa que nós tivemos sucesso ao criar o objeto
type(soup)

### ✍️ Faça você mesmo

Use os blocos de código abaixo para experimentar a busca de HTML e transformá-lo em sopa (se você buscou algumas páginas anteriormente e as salvou como variáveis, seria um bom começo).

### Segmentação e extração de dados

Agora que temos o objeto BeautifulSoup carregado, podemos procurar os elementos HTML específicos que contêm os dados de que precisamos. Nossa estratégia geral:
1. Encontre a tabela principal com os dados que queremos pegar
2. Obtenha uma lista de linhas (o elemento `tr`, que significa "table row") naquela tabela
3. Use um Python `for loop` para percorrer cada linha da tabela e encontrar os dados dentro dela (`td`, ou "table data")

Para fazer isso, usaremos dois métodos `bs4`:
- [`find()`](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find), que retorna o primeiro elemento que corresponde aos critérios que você deseja
- [`find_all()`](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find-all), que retona uma _lista_ de elementos que correspondem aos critérios. ([Veja como as listas Python funcionam](https://github.com/abraji/cursos_NICAR20/blob/master/python_ire/Python%20101.ipynb).)

#### Encontre a tabela

Para começar, precisamos encontrar a tabela. Existem várias maneiras de conseguir isso, mas porque esta é a única tabela na página (ver fonte e pressione `Ctrl+F` para procurar por `<table` para confirmar), podemos simplesmente dizer: "Olhe através do objeto `soup` e encontre a tag da tabela ".

Traduzido, o código é: `soup.find('table')`. Enquanto estamos nisso, salve os resultados dessa pesquisa em uma nova variável, `table`.

Execute essas células:

In [ ]:
table = soup.find('table')

In [ ]:
table

#### Encontre as linhas na tabela

Próximo, use o método `find_all()` para detalhar e obter uma lista de linhas na tabela:

In [ ]:
rows = table.find_all('tr')

In [ ]:
rows

Para ver quantos itens estão nessa lista - em outras palavras, quantas linhas estão na tabela - você pode usar a função `len()`:

In [ ]:
len(rows)

#### Faça um loop pelas linhas e extraia os dados

Em seguida, podemos usar um [`for` loop](https://github.com/abraji/cursos_NICAR20/blob/master/python_ire/Python%20101.ipynb) para percorrer a lista de linhas e começar a coletar dados de cada uma.

Atualização rápida sobre a sintaxe do _for loop_: Comece com a palavra `for` (minúsculo), um nome de variável para cada item da lista em que você está repetindo, e a palavra `in` (minúsculo), então o nome da lista que contém os itens (`rows`, no nosso caso), depois o sinal dois pontos, depois um bloco de código recuado que descreve o que estamos fazendo com cada item da lista.

Cada dado da linha será armazenado em uma tag `td`, que significa "table data." Então, dentro do loop - no bloco recuado - usaremos o método `find_all()` para obter uma lista de todas as tags `td` dentro da linha. E a partir daí, podemos acessar o conteúdo dentro de cada tag.

Nosso objetivo é terminar com uma _lista_ de dados para cada linha que eventualmente gravaremos em um arquivo. Normalmente, você provavelmente faria o trabalho de repetir e inspecionar os resultados, passo a passo, em uma célula de código. Mas para mostrar o pensamento de como você pode abordar isso (e praticar a sintaxe), começaremos apenas imprimindo cada linha e construindo a partir daí. (`print('='*80)` imprimirá uma linha de 80 sinais de igual - uma maneira de nos ajudar a ver exatamente com o que estamos trabalhando em cada linha.)

In [ ]:
for row in rows:
    print(row)
    print('='*80)

Observe que o primeiro item impresso é a linha do cabeçalho com os rótulos da coluna. Você é livre para manter esses cabeçalhos, se quiser, mas normalmente pulo essa linha e defino minha própria lista de nomes de colunas.

(Outra coisa a considerar: em páginas da Web melhor construídas, as células na linha do cabeçalho serão representadas por tags `th` ("table header"), não tags `td` ("table data"). O próximo passo do nosso loop `for` é, "Procurar todas as tags `td` na linha," isso será algo com o qual você precisará lidar.)

Podemos pular a primeira linha usando _fatiamento de lista_: adicionando colchetes após o nome da lista com algumas instruções sobre quais itens da lista queremos selecionar.

Aqui, a sintaxe seria: `rows[1:]`, que significa, pegue tudo na lista `rows` começando com o item na posição 1 (o segundo item) até o final da lista. Como muitas linguagens de programação, o Python começa a contar em 0, então o resultado deixa o primeiro item da lista - ou seja, o item na posição 0, ou seja, os cabeçalhos.

In [ ]:
for row in rows[1:]:
    print(row)
    print('='*80)

Agora estamos cozinhando com gás. Vamos começar a extrair os dados em cada linha. Comece usando `find_all()` para pegar uma lista de tags `td`:

In [ ]:
for row in rows[1:]:
    cells = row.find_all('td')
    print(cells)
    print('='*80)

Agora temos, para cada linha, uma _lista_ de tags `td`. O próximo passo é olhar para a tabela e começar a pegar valores específicos com base em sua posição na lista e atribuí-los a nomes de variáveis legíveis por humanos.

Lembrança rápida da sintaxe da lista: Para acessar um item específico em uma lista, use colchetes `[]` e o número do índice do item que você deseja acessar. Por exemplo, para obter a primeira célula da linha - a data em que cada relatório WARN foi emitido -- use `[0]`.

In [ ]:
for row in rows[1:]:
    cells = row.find_all('td')
    warn_date = cells[0]
    print(warn_date)
    print('='*80)

Isso está retornando todo o objeto `Tag` -- nós apenas queremos o conteúdo dentro dele. Você pode acessar o atributo `.text` da tag para inserir o texto:

In [ ]:
for row in rows[1:]:
    cells = row.find_all('td')
    warn_date = cells[0].text    
    print(warn_date)

Na próxima célula (`[1]`), o atributo `.text` fornecerá o código NAICS. Na terceira célula (`[2]`) você receberá o nome da empresa. Etc.

Também é geralmente uma boa prática reduzir o espaço em branco externo para cada valor, e você pode usar o método de string interno do Python `strip()` para fazer isso enquanto você caminha pela linha.

O que nos leva até aqui:

In [ ]:
for row in rows[1:]:
    cells = row.find_all('td')
    warn_date = cells[0].text.strip()
    naics_code = cells[1].text.strip()
    biz = cells[2].text.strip()
    print(warn_date, naics_code, biz)

### ✍️ Faça você mesmo

Agora que você chegou até aqui, veja se consegue isolar os outros dados em cada linha.

In [ ]:
for row in rows[1:]:
    cells = row.find_all('td')
    warn_date = cells[0].text.strip()
    naics_code = cells[1].text.strip()
    biz = cells[2].text.strip()
    
    # address
    
    # wia_code
    
    # total_employees
    
    # effective_date
    
    # type_code

    # print()

### Escreva os resultados no arquivo

Agora que direcionamos nossas listas de dados para cada linha, podemos usar o recurso interno do Python, o módulo [`csv`](https://docs.python.org/3/library/csv.html) para escrever cada lista em um arquivo CSV.

Primeiro, importe o módulo csv.

In [6]:
import csv

Agora defina uma lista de cabeçalhos para corresponder aos dados (cada cabeçalho de coluna será uma string) - execute esta célula:

In [7]:
HEADERS = ['warn_date', 'naics_code', 'biz', 'address', 'wia_code',
           'total_employees', 'effective_date', 'type_code']

Agora, usando algo chamado um bloco `with`, abra um novo arquivo CSV para gravar e escreva algum código para fazer o seguinte:
- Crie um objeto `csv.writer`
- Escreva a lista de cabeçalhos usando o método `writerow()` do objeto `csv.writer` 
- Volte para o `for` loop que você acabou de escrever e, em vez de apenas imprimir o conteúdo de cada célula, crie uma lista de itens e use o método `writerow()` do objeto `csv.writer` para escrever sua lista de dados em arquivo

In [ ]:
# crie um arquivo chamado 'warn-data.csv' no modo de escrita ('w') 
# especifique que novas linhas são encerradas por uma sequência vazia (isso lida com um problema específico do PC)
# e use a palavra-chave `as` para nomear o manipulador de arquivo aberto (o nome da variável `outfile` pode ser qualquer outro)
with open('warn-data.csv', 'w', newline='') as outfile:
    # vá para o módulo csv que importamos e crie um novo objeto .writer anexado ao arquivo aberto
    # e salve-o em uma variável
    writer = csv.writer(outfile)

    # escreva a lista de cabeçalhos
    writer.writerow(HEADERS)
    
    # cole no loop for que você escreveu anteriormente aqui - cuidado com a identação!
    # deve estar nesse nível de indentação =>
    # for row in rows[1:]:
    #     cells = row.find_all('td')
    #     etc. ...
    # mas no final, em vez de `print(warn_date, naics_code, ...etc.)`
    # faça algo como
    # data_out = [warn_date, naics_code, ...etc.]
    # `writer.writerow(data_out)`

Se você procurar na pasta, verá um novo arquivo: `warn-data.csv`. Hooray!

🎉 🎉 🎉

### ✍️ Faça você mesmo

Juntando tudo:
- Encontre um site que você deseja copiar
- Use `requests` buscar o HTML
- Use `bs4` para analisar o HTML e isolar os dados nos quais você está interessado
- Use `csv` para gravar os dados no arquivo

### Extra credit problems

1. **Remover espaço em branco interno:** Ao examinar os dados, você provavelmente notou que alguns dos valores têm algum espaço em branco interno desnecessário, que você pode corrigir antes de gravar cada linha no arquivo. O Python não possui um método de string interno para remover o espaço em branco interno, infelizmente, mas [procurando no Google](https://www.google.com/search?q=python+remove+internal+whitespace) vimos uma estratégia comum: usar o método `split()` para separar palavras individuais, na sequência `join()` (una) a lista resultante em um único espaço. Como um exemplo:

```python
my_text = 'hello     world      how are      you?'

# split() vai transformar isso em uma lista de palavras
my_text_words = my_text.split()
# ['hello', 'world', 'how', 'are', 'you?']

# juntar-se em um único espaço
my_text_clean = ' '.join(my_text_words)
print(my_text_clean)
# prints 'hello world how are you?'

# ou, como uma linha
my_text_clean = ' '.join(my_text.split())
```

2. **Buscar vários anos:** A tabela que raspamos possui WARN de aviso para o ano atual, mas a agência também mantém páginas com avisos de aviso para anos anteriores-- há uma lista deles em uma seção [na parte inferior da página](https://www.dllr.state.md.us/employment/warn.shtml). Veja se você pode descobrir como fazer um loop em várias páginas e juntar o conteúdo de cada uma em um único CSV.


3. **Construa uma tabela de pesquisa:** Cada código numérico na coluna "WIA Code" corresponde a uma área local. Veja se você consegue descobrir como criar um dicionário de pesquisa que mapeie os números para seus locais e, ao percorrer a tabela de dados, substitua o valor numérico nessa coluna pelo nome da área local. Aqui está uma dica:

```python
    lookup_dict = {
        '1': 'hello',
        '2': 'world'
    }

    print(lookup_dict.get('1'))
    # prints 'hello'

    print(lookup_dict.get('3'))
    # prints None

```


4. **Corrigir erros de codificação (acentuação):** Você deve ter notado alguns problemas de codificação -- ex., `Nestlé` está sendo renderizado como `NestlÃ©`. Isso ocorre devido a um problema de codificação -- o `warn_page.text` não está codificado como`utf-8`. Ao usar `decode()` e `encode()`, veja se você pode consertar isso. (Dica! Parece que o estado de Maryland é um grande fã de `latin-1`.)